In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import os

In [2]:
# 다운로드 폴더 설정
if not os.path.isdir('./../../Data/daily_price'):
    os.mkdir('./../../Data/daily_price')
os.chdir('./../../Data/daily_price')

if not os.path.isdir('./../daily_price/kospi'):
    os.mkdir('./../daily_price/kospi')
if not os.path.isdir('./../daily_price/kosdaq'):
    os.mkdir('./../daily_price/kosdaq')

In [6]:
def daily_price(code:str):
    page=1

    url='https://finance.naver.com/item/sise_day.nhn?code='+ code + '&page='

    # driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url+str(page))
    html = bs(driver.page_source, 'html.parser')
    endPage=html.find('td', class_="pgRR").find('a')['href'].split('=')[-1]

    nav_finance=[]

    for page in range(1, int(endPage)+1):
        driver.get(url+str(page))
        html = bs(driver.page_source, 'html.parser')
        soup_tr=html.find_all('tr')
        for i in range(2, 15):
            day_data=[]
            soup_td=soup_tr[i].find_all('td')
            if len(soup_td) < 7 :
                continue
            else:
                for td_txt in soup_td:
                    if td_txt.text == u'\xa0':
                        continue
                    else:
                        day_data.append(td_txt.text.replace('.', '-').replace('\n','').replace('\t', '').replace(',', ''))
                        # day_data.append(re('[가-힣0-9a-zA-Z]+',td_txt.text))

            nav_finance.append(day_data)

    driver.close()

    df=pd.DataFrame(nav_finance, columns=['날짜','종가',"전일비",'시가','고가','저가','거래량'])
    # print(df.info())
    df=df.dropna()
    # df['날짜']=pd.to_datetime(df['날짜'])
    df['날짜'] = [ pd.to_datetime(df.iloc[date, 0]).strftime('%Y%m%d') for date in range(len(df))]
    df = df.sort_values(by='날짜')
    df.reset_index(inplace=True,drop=True)

    # 전일비 상승,하락 계산
    for idx in df.index[1:]:
        if df.loc[idx,'종가'] < df.loc[idx-1,'종가'] :
            df.loc[idx,'전일비'] = int(-int(df.loc[idx,'전일비']))

    # 전일비 비율 생성
    df['전일비_비율'] = 0
    for idx in df.index[1:]:
        df.loc[idx,'전일비_비율'] = int(df.loc[idx,'전일비']) / int(df.loc[idx-1,'종가']) * 100

    df['전일비_비율'] = df['전일비_비율'].apply(lambda x : str(round(x, 2)) + '%')


    # 파일 저장 코드
    # 코스피 폴더
    # df.to_csv(f'./kospi/{code}_dailyprice.csv', index=False,encoding='utf-8-sig')
    # 코스닥 폴더
    df.to_csv(f'./kosdaq/{code}_dailyprice.csv', index=False,encoding='utf-8-sig')


In [4]:
# 코스피 10개
kospi_list = ["005380","035420","068270","005930","000270","373220","207940","000660","006400","051910"]

# 코스닥 10개
kosdaq_list = ["091990","247540","066970","263750","293490","112040","028300","278280","035760","058470"]

In [5]:
# for i in kospi_list:
#     daily_price(i)

In [7]:
for i in kosdaq_list:
    daily_price(i)